# Building `gradio` demos within `fastai`

## Vision example 

In [ ]:
from fastai.vision.all import *
import gradio as gr

First we load a `learner`

In [ ]:
path = untar_data(URLs.PETS)
Path.BASE_PATH = path
fname = (path/"images").ls()[0]

pets = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 splitter=RandomSplitter(seed=42),
                 get_y=using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'),
                 item_tfms=Resize(460),
                 batch_tfms=aug_transforms(size=224, min_scale=0.75))
dls = pets.dataloaders(path/"images")

learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn = learn.load('pets')

Now that the learner is ready we can immediately create a default demo. Without any parameters `demo` will read the type of `inputs` and `outputs` and create a barebones `Interface`

In [ ]:
learn.demo()

Running locally at: http://127.0.0.1:7863/
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://43744.gradio.app
Interface loading below...


You can also customize your demo by adding any of the parameters we support in the gradio `Interface` class (see [gradio.app/docs](gradio.app/docs)) directly into `demo`. For example below we'll add a title, description, and examples  to this interface. 

In [ ]:
title="Pet classification" 
description="Classify pets into 1 of 37 classes"
# downloading an image to use as an example
urlretrieve("https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/golden-retriever-royalty-free-image-506756303-1560962726.jpg","dog.jpg")
examples = [["dog.jpg"]]

# simply pass in the new parameters you want to add to demo
learn.demo(title=title, description=description, examples=examples)


Running locally at: http://127.0.0.1:7864/
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://13354.gradio.app
Interface loading below...


You can also override the parameters that are inferred from `learner` (which are the `predict_fn`, `inputs`, and `outputs`) or set to `True` by default (`share` which creates an external url, `allow_screenshot` which adds a screenshot button, and `allow_flagging` which adds a flag button) 

We will change all of those below. 

In [ ]:
# defining a new predict function that takes in an extra input (checkbox)
def new_predict(image, remove_underscores):
    preds = learn.predict(image.name)
    output = {learn.dls.vocab[i]: float(preds[2][i]) for i in range(len(learn.dls.vocab))}
    if remove_underscores: 
        output = {key.replace("_", " "): value for key, value in zip(output.keys(), output.values())}
    return output

# defining a list of inputs to add the checkbox 
inputs = [gr.inputs.Image(type="file", label="image of pet"), gr.inputs.Checkbox(label="remove underscores?")]
examples = [["dog.jpg", True]]


# defining a new output and increasing the top classes to 5 
outputs = gr.outputs.Label(num_top_classes=5, label="breed")

# setting share = False to only run this locally 
share = False 

# no flagging, and no screenshot button 
allow_flagging = False; allow_screenshot = False 

# putting everything into demo 
learn.demo(predict_fn=new_predict, inputs=inputs, outputs=outputs, 
           share=share, allow_flagging=allow_flagging, allow_screenshot=allow_screenshot, 
           title=title, description=description, examples=examples)

Running locally at: http://127.0.0.1:7865/
To create a public link, set `share=True` in `launch()`.
Interface loading below...
